In [96]:
import numpy as np
import pandas as pd

In [97]:
config = np.genfromtxt("./cnf_sv.csv", dtype=str, delimiter=",")
config[0][1]

'2000'

In [98]:
tamaño_train = int(config[0][1])
tamaño_test = int(config[1][1])
valor_relevancia = float(config[2][1])
vectores_singulares = int(config[3][1])
clase_1 = bool(int(config[4][1]))
clase_2 = bool(int(config[5][1]))
clase_3 = bool(int(config[6][1]))

In [99]:
data_train = np.genfromtxt("./Archivos/KDDTrain.csv", dtype=str, delimiter=",")
data_train

array([['0', 'tcp', 'ftp_data', ..., '0.00', 'normal', '20'],
       ['0', 'udp', 'other', ..., '0.00', 'normal', '15'],
       ['0', 'tcp', 'private', ..., '0.00', 'neptune', '19'],
       ...,
       ['0', 'tcp', 'private', ..., '1.00', 'neptune', '21'],
       ['0', 'tcp', 'nnsp', ..., '0.00', 'neptune', '20'],
       ['0', 'tcp', 'finger', ..., '0.00', 'neptune', '18']], dtype='<U15')

In [100]:
clases = {  
          'normal':1, 
          'neptune':2, 'teardrop':2, 'smurf':2, 'pod':2, 'back':2,
          'land':2, 'apache2':2,'processtable':2,'mailbomb':2, 'udpstorm':2,
          'ipsweep':3, 'portsweep':3, 'nmap':3, 'satan':3, 'saint':3, 'mscan':3
       }

In [101]:
contador = 0
for row in data_train[:, 41]:
    if row in clases:
        data_train[contador, 41] = clases[row]
    else:
        data_train = np.delete(data_train, contador, axis=0)
        contador = contador - 1
    contador = contador + 1

In [102]:
data_train[:, 41]

array(['1', '1', '2', ..., '2', '2', '2'], dtype='<U15')

In [103]:
nuevo_data_train = []
cantidad_clase_1 = 0
cantidad_clase_2 = 0
cantidad_clase_3 = 0
fila = 0

for row in data_train[:, 41]:
    if cantidad_clase_1 == 0 and cantidad_clase_2 == 0 and cantidad_clase_3 == 0:
        if clase_1 and row == '1':
            nuevo_data_train = [data_train[fila]]
            cantidad_clase_1= cantidad_clase_1 + 1
        elif clase_2 and row == '2':
            nuevo_data_train = [data_train[fila]]
            cantidad_clase_2= cantidad_clase_2 + 1
        elif clase_3 and row == '3':
            nuevo_data_train = [data_train[fila]]
            cantidad_clase_3= cantidad_clase_3 + 1
    elif(clase_1 and row == '1' and cantidad_clase_1 < tamaño_train):
        nuevo_data_train = np.append(nuevo_data_train, [data_train[fila]], axis=0)
        cantidad_clase_1= cantidad_clase_1 + 1
    elif(clase_2 and row == '2' and cantidad_clase_2 < tamaño_train):
        nuevo_data_train = np.append(nuevo_data_train, [data_train[fila]], axis=0)
        cantidad_clase_2= cantidad_clase_2 + 1
    elif(clase_3 and row == '3' and cantidad_clase_3 < tamaño_train):
        nuevo_data_train = np.append(nuevo_data_train, [data_train[fila]], axis=0)
        cantidad_clase_3= cantidad_clase_3 + 1
    fila = fila + 1
nuevo_data_train

array([['0', 'tcp', 'ftp_data', ..., '0.00', '1', '20'],
       ['0', 'udp', 'other', ..., '0.00', '1', '15'],
       ['0', 'tcp', 'http', ..., '0.01', '1', '21'],
       ...,
       ['0', 'icmp', 'eco_i', ..., '0.00', '3', '16'],
       ['0', 'udp', 'other', ..., '0.00', '3', '10'],
       ['0', 'icmp', 'eco_i', ..., '0.00', '3', '15']], dtype='<U15')

In [104]:
def one_hot(array):
    dic = {}
    for i, cosa in enumerate(np.unique(array)):
        dic[cosa] = i
    return dic

In [105]:
protocolos = []
servicios = []
flags = []

protocolo_unico = []
servicio_unico = []
flag_unico = []

for row in nuevo_data_train:
    protocolos.append(row[1])
    servicios.append(row[2])
    flags.append(row[3])
    
protocolo = one_hot(np.unique(protocolos))
servicio = one_hot(np.unique(servicios))
flag = one_hot(np.unique(flags))

In [106]:
contador = 0
for row in nuevo_data_train:
    nuevo_data_train[contador, 1] = protocolo[row[1]]
    nuevo_data_train[contador, 2] = servicio[row[2]]
    nuevo_data_train[contador, 3] = flag[row[3]]
    contador = contador + 1

In [107]:
nuevo_data_train

array([['0', '1', '19', ..., '0.00', '1', '20'],
       ['0', '2', '40', ..., '0.00', '1', '15'],
       ['0', '1', '22', ..., '0.01', '1', '21'],
       ...,
       ['0', '0', '13', ..., '0.00', '3', '16'],
       ['0', '2', '40', ..., '0.00', '3', '10'],
       ['0', '0', '13', ..., '0.00', '3', '15']], dtype='<U15')

In [108]:
nuevo_data_train = np.delete(nuevo_data_train, 42, 1)

In [109]:
for i in range(0, 41):
    min = 999999
    max = 0
    for j in nuevo_data_train[:,i]:
        if float(j) < min:
            min = float(j)
        if float(j) > max:
            max = float(j)
    a = 0.01
    b = 0.99
    fila = 0
    if min != max:
        for x in nuevo_data_train[:, i]:
            value = ((float(x) - min)/(max - min))*(b-a)+a
            nuevo_data_train[fila,i] = value
            fila = fila + 1
            
print(nuevo_data_train)


[['0.01' '0.5' '0.3203333333333' ... '0.0590000000000' '0.01' '1']
 ['0.01' '0.99' '0.6633333333333' ... '0.01' '0.01' '1']
 ['0.01' '0.5' '0.3693333333333' ... '0.01' '0.0197999999999' '1']
 ...
 ['0.01' '0.01' '0.2223333333333' ... '0.01' '0.01' '3']
 ['0.01' '0.99' '0.6633333333333' ... '0.01' '0.01' '3']
 ['0.01' '0.01' '0.2223333333333' ... '0.01' '0.01' '3']]


In [110]:
print(nuevo_data_train[0])

['0.01' '0.5' '0.3203333333333' '0.892' '0.0100012605935' '0.01' '0' '0'
 '0' '0.01' '0.01' '0.01' '0.01' '0.01' '0.01' '0.01' '0.01' '0' '0.01'
 '0' '0' '0.01' '0.0119215686274' '0.0119797979797' '0.01' '0.01' '0.01'
 '0.01' '0.99' '0.01' '0.01' '0.5848818897637' '0.1025984251968' '0.1766'
 '0.0394' '0.1766' '0.01' '0.01' '0.01' '0.0590000000000' '0.01' '1']


In [111]:
n_clases = int(config[4][1]) + int(config[5][1]) + int(config[6][1])
clases,d = np.unique(nuevo_data_train[:,41],return_counts=True)
p = d/np.sum(d)
I = -(np.sum(p*np.log2(p)))

In [112]:
print(clases)
print(d)
print(p)
print(I)

['1' '3']
[2000 2000]
[0.5 0.5]
1.0


In [113]:
def count(data, i, nuevo_data_train, n_clases):
    elementos_clase = []
    for row in nuevo_data_train:
        print("jola")

In [114]:
def particion(data, i):
    d_ij = []
    for element in np.unique(data[:, i]):
        particion = []
        for j in range(0, data.shape[0]):
            if data[j][i] == element:
                particion.append(data[j])
        d_ij.append(contador_clases(np.array(particion)))
    return np.array(d_ij)

def contador_clases(particion):
    cont_clase1 = 0
    cont_clase2 = 0
    cont_clase3 = 0
    #print(particion.shape[0])
    for i in range(0, particion.shape[0]):
        if particion[i, 41] == '1':
            cont_clase1 = cont_clase1 + 1
        if particion[i, 41] == '2':
            cont_clase2 = cont_clase2 + 1
        if particion[i, 41] == '3':
            cont_clase3 = cont_clase3 + 1
    return np.array([cont_clase1, cont_clase2, cont_clase3])

In [115]:
IG = []
for i in range(0, 41):
    # data,d = np.unique(nuevo_data_train[:, i], return_counts=True)
    # #p = d/np.sum(d)
    # I_x = -(np.sum(p*np.log2(p)))
    # E = np.sum(d/np.sum(d)*I_x)
    #print('I_x: ',I_x,' E: ', E)
    #IG.append(I - E)
    d_ij = particion(nuevo_data_train, i)
    E_x = 0
    I_d = 0
    for d in d_ij:
        p = np.sum(d)/nuevo_data_train.shape[0]
        N = np.sum(d)
        for d_ in d:
            if d_ != 0:
                I_d = -np.sum((d_/N)*np.log2(d_/N))
        E_x = E_x + np.sum(p*I_d)
    i_g = I - E_x
    IG.append(i_g)
np.array(IG)
print(IG)
            
        


[0.5330274636298231, 0.5592230009447783, 0.9141080554692964, 0.5802343089065811, 0.9536894356344656, 0.8836444197408198, 0.5, 0.5, 0.5, 0.5043667845990943, 0.5006107640038666, 0.8004590262695201, 0.5023481973323782, 0.500902248423374, 0.5003607639587296, 0.5032533894707922, 0.500575467990322, 0.5, 0.5021672951319456, 0.5, 0.5, 0.502422911122758, 0.6303891996568334, 0.5795698302788455, 0.529974196314414, 0.5122095510541739, 0.5767162584630632, 0.5733173140086938, 0.5534192750407759, 0.5519491383821047, 0.6316417167284392, 0.6401652613314613, 0.748777369570545, 0.6384215770479303, 0.6261197526320279, 0.6751288859430122, 0.7214546006309728, 0.534513936494144, 0.5316329299263853, 0.593777812727958, 0.5944735498428293]


In [116]:
a = np.array(IG)

ind = np.argpartition(a, -int(valor_relevancia))[-int(valor_relevancia):]

top4 = a[ind]

ind_sort = ind[np.argsort(a[ind])][::-1]
a_sort = a[ind[np.argsort(a[ind])]][::-1]

print(top4)

[0.55341928 0.559223   0.57331731 0.59377781 0.58023431 0.57671626
 0.57956983 0.59447355 0.95368944 0.63164172 0.64016526 0.74877737
 0.63842158 0.62611975 0.67512889 0.7214546  0.6303892  0.91410806
 0.88364442 0.80045903]


In [117]:
print(ind_sort)
print(a_sort)

[ 4  2  5 11 32 36 35 31 33 30 22 34 40 39  3 23 26 27  1 28]
[0.95368944 0.91410806 0.88364442 0.80045903 0.74877737 0.7214546
 0.67512889 0.64016526 0.63842158 0.63164172 0.6303892  0.62611975
 0.59447355 0.59377781 0.58023431 0.57956983 0.57671626 0.57331731
 0.559223   0.55341928]


In [118]:
np.savetxt('index.csv', ind_sort, fmt='%.0f')

In [119]:
data_index = nuevo_data_train[:, ind_sort]
data_index = data_index.astype(np.float32)
data_index

array([[0.01000126, 0.32033333, 0.01      , ..., 0.01      , 0.5       ,
        0.99      ],
       [0.01000038, 0.66333336, 0.01      , ..., 0.01      , 0.99      ,
        0.0884    ],
       [0.0100006 , 0.36933333, 0.03920887, ..., 0.01      , 0.5       ,
        0.99      ],
       ...,
       [0.01000002, 0.22233333, 0.01      , ..., 0.01      , 0.01      ,
        0.99      ],
       [0.01000001, 0.66333336, 0.01      , ..., 0.01      , 0.99      ,
        0.99      ],
       [0.01000002, 0.22233333, 0.01      , ..., 0.01      , 0.01      ,
        0.99      ]], dtype=float32)

In [120]:
#nuevo_data_frame = pd.DataFrame(nuevo_data_train)
#data_index = nuevo_data_frame[ind_sort]
#data_index

In [121]:
for i in range(0, int(valor_relevancia)):
    x_i = data_index[:, i].mean()
    data_index[:, i] = data_index[:, i] - x_i
data_index
wea2 = pd.DataFrame(data_index)
wea2


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.000265,-0.147960,-0.005044,-0.355005,-0.349223,-0.105134,-0.216793,0.018272,-0.419286,-0.205433,-0.090421,-0.187121,-0.236736,-0.165159,0.168707,-0.031642,-0.236231,-0.24045,0.017028,0.160007
1,-0.000266,0.195040,-0.005044,-0.355005,-0.441822,-0.105134,0.479007,0.423390,-0.585886,-0.205433,-0.069284,0.371479,-0.236736,-0.214159,0.168707,-0.033622,-0.236231,-0.24045,0.507028,-0.741593
2,-0.000266,-0.098960,0.024165,0.624995,0.538178,-0.065934,-0.353993,-0.444720,0.394114,-0.205433,-0.084656,-0.216521,-0.226936,-0.214159,0.168707,-0.025703,-0.236231,-0.24045,0.017028,0.160007
3,-0.000266,-0.098960,-0.003539,0.624995,0.538178,-0.105134,-0.383393,0.423390,0.394114,-0.117233,-0.036617,-0.216521,-0.236736,-0.214159,0.168707,0.027752,-0.236231,-0.24045,0.017028,0.160007
4,-0.000265,-0.098960,0.003021,0.624995,0.399281,-0.075734,-0.265793,-0.529602,0.394114,0.215967,-0.088499,-0.216521,-0.236736,-0.214159,0.168707,-0.021743,-0.236231,-0.24045,0.017028,0.160007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,-0.000266,0.260374,-0.005044,-0.355005,-0.437964,0.874866,-0.363793,-0.340547,-0.566286,-0.205433,-0.092342,0.763479,0.743264,0.726641,-0.615293,-0.033622,0.743769,0.73955,0.017028,0.160007
3996,-0.000266,0.260374,-0.005044,-0.355005,-0.441822,-0.105134,0.008607,-0.251807,-0.576086,-0.205433,-0.092342,0.185279,0.743264,0.158241,-0.321293,-0.033622,0.743769,0.73955,0.017028,0.160007
3997,-0.000266,-0.245960,-0.005044,-0.355005,-0.206467,0.404466,0.596607,-0.552752,0.394114,0.774567,-0.092342,-0.216521,-0.236736,-0.214159,0.168707,0.055469,-0.236231,-0.24045,-0.472972,0.160007
3998,-0.000266,0.195040,-0.005044,-0.355005,-0.418672,-0.105134,-0.353993,0.423390,-0.556486,-0.205433,-0.092342,-0.196921,-0.236736,-0.214159,0.168707,-0.033622,-0.236231,-0.24045,0.507028,0.160007


In [122]:
data_index_t = np.transpose(data_index)
ye = data_index.T/np.sqrt(data_index.shape[0] - 1)


In [123]:
print(ye)

[[-4.1896642e-06 -4.2036700e-06 -4.2001793e-06 ... -4.2092811e-06
  -4.2093989e-06 -4.2092811e-06]
 [-2.3397394e-03  3.0842451e-03 -1.5648847e-03 ... -3.8894492e-03
   3.0842451e-03 -3.8894492e-03]
 [-7.9760910e-05 -7.9760910e-05  3.8212963e-04 ... -7.9760910e-05
  -7.9760910e-05 -7.9760910e-05]
 ...
 [-3.8023288e-03 -3.8023288e-03 -3.8023288e-03 ... -3.8023288e-03
  -3.8023288e-03 -3.8023288e-03]
 [ 2.6926250e-04  8.0178110e-03  2.6926250e-04 ... -7.4792863e-03
   8.0178110e-03 -7.4792863e-03]
 [ 2.5302519e-03 -1.1727078e-02  2.5302519e-03 ...  2.5302519e-03
   2.5302519e-03  2.5302519e-03]]


In [124]:
data_index.shape[0]

4000

In [93]:
wea = pd.DataFrame(ye)
wea

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,...,-4.209281e-06,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004
1,-0.002340,0.003084,-0.001565,-0.001565,-0.001565,-0.001565,-0.003889,-0.001565,-0.001565,-0.001565,...,-3.889449e-03,-0.003889,0.003084,0.003084,0.004117,0.004117,0.004117,-0.003889,0.003084,-0.003889
2,-0.000080,-0.000080,0.000382,-0.000056,0.000048,0.000701,-0.000080,-0.000045,-0.000013,0.000595,...,-7.976091e-05,-0.000080,-0.000080,-0.000080,-0.000080,-0.000080,-0.000080,-0.000080,-0.000080,-0.000080
3,-0.005614,-0.005614,0.009883,0.009883,0.009883,0.009883,-0.005614,0.009883,0.009883,0.009883,...,-5.613824e-03,-0.005614,-0.005614,-0.005614,-0.005614,-0.005614,-0.005614,-0.005614,-0.005614,-0.005614
4,-0.005522,-0.006987,0.008510,0.008510,0.006314,0.008510,-0.006072,0.008510,0.008510,0.008510,...,-3.508994e-03,0.004850,-0.006926,-0.006926,0.003934,-0.006926,-0.006987,-0.003265,-0.006621,0.006680
5,-0.001663,-0.001663,-0.001043,-0.001663,-0.001198,-0.001353,0.013835,-0.001198,-0.001043,-0.001353,...,2.366717e-03,0.002212,-0.001663,-0.001663,-0.001663,0.013835,-0.001663,0.006396,-0.001663,0.006086
6,-0.003428,0.007575,-0.005598,-0.006063,-0.004203,-0.005908,0.009434,-0.005753,-0.005908,-0.005908,...,9.434362e-03,0.009434,0.009434,0.002616,0.009434,-0.005753,0.000136,0.009434,-0.005598,0.009434
7,0.000289,0.006695,-0.007033,0.006695,-0.008375,-0.003311,-0.008802,-0.004836,0.000716,-0.003555,...,-8.740867e-03,-0.008680,0.006695,0.006695,0.006695,-0.005385,-0.003982,-0.008741,0.006695,-0.008802
8,-0.006630,-0.009265,0.006232,0.006232,0.006232,0.006232,0.006232,0.006232,0.006232,0.006232,...,6.232274e-03,0.006232,-0.009110,-0.009110,0.001738,-0.008955,-0.009110,0.006232,-0.008800,0.006232
9,-0.003249,-0.003249,-0.003249,-0.001854,0.003415,0.000161,-0.003249,-0.003249,-0.000149,-0.000149,...,1.224852e-02,0.012249,-0.003249,-0.003249,-0.003249,-0.003249,-0.003249,0.012249,-0.003249,0.012249


In [130]:
U,S,VT = np.linalg.svd(ye,full_matrices=False)
print((VT).shape)


(20, 4000)


In [126]:
data_final = np.dot(VT[:, :vectores_singulares].T, data_index.T).T
print(data_final)


[[-0.00040192 -0.01105213  0.00503226 ...  0.02717827 -0.00042696
  -0.00887664]
 [-0.01289967  0.01210778  0.01795359 ...  0.00223697  0.03518344
  -0.01220054]
 [ 0.01530139 -0.02118279 -0.00965462 ...  0.00964414 -0.02327213
   0.00572539]
 ...
 [ 0.00523988 -0.01888149  0.00093453 ...  0.01170024 -0.00661007
   0.00491328]
 [-0.00149346 -0.01847807  0.0023411  ...  0.03494783  0.00493651
  -0.01276058]
 [ 0.00505281 -0.01397233  0.00216245 ...  0.01059198 -0.00617585
   0.00048147]]
